In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json


In [2]:
!pip install simplet5 -q

In [4]:
import pandas as pd

# Update the file path based on your directory structure
file_path = "../input/arxiv/arxiv-metadata-oai-snapshot.json"

# Define the chunk size (number of lines to read at a time)
chunk_size = 10000  # Adjust this value based on your available memory

# Initialize an empty list to store DataFrames
dfs = []

# Read and process the JSON file in chunks
for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    # Select specific columns from the chunk
    chunk = chunk[['title', 'abstract']]
    
    # Append the chunk to the list of DataFrames
    dfs.append(chunk)

# Concatenate the list of DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)



In [5]:
df.head()

,title,abstract
0,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...


In [7]:
df=df.rename(columns={"title":"target_text","abstract":"source_text"})

In [8]:
df=df[['source_text','target_text']]

In [9]:
df.head()

,source_text,target_text
0,A fully differential calculation in perturba...,Calculation of prompt diphoton production cros...
1,"We describe a new algorithm, the $(k,\ell)$-...",Sparsity-certifying Graph Decompositions
2,The evolution of Earth-Moon system is descri...,The evolution of the Earth-Moon system based o...
3,We show that a determinant of Stirling cycle...,A determinant of Stirling cycle numbers counts...
4,In this paper we show how to compute the $\L...,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...


In [10]:
df['source_text']="summarize: "+df['source_text']

In [11]:
df.head()

,source_text,target_text
0,summarize: A fully differential calculation ...,Calculation of prompt diphoton production cros...
1,"summarize: We describe a new algorithm, the ...",Sparsity-certifying Graph Decompositions
2,summarize: The evolution of Earth-Moon syste...,The evolution of the Earth-Moon system based o...
3,summarize: We show that a determinant of Sti...,A determinant of Stirling cycle numbers counts...
4,summarize: In this paper we show how to comp...,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...


In [12]:
from sklearn.model_selection import train_test_split


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
train_df,test_df=train_test_split(df,test_size=0.3)
train_df.shape, test_df.shape


((1612737, 2), (691174, 2))

In [16]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-io


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 34.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstall

In [17]:
from simplet5 import SimpleT5
model=SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

In [18]:
model.train(train_df=train_df[:5000],
           eval_df=test_df[:100],
           source_max_token_len=128,
           target_max_token_len=50,
           batch_size=8,
           max_epochs=5,
           use_gpu=True)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [19]:
! (cd outputs; ls)

simplet5-epoch-0-train-loss-2.3749-val-loss-2.1523
simplet5-epoch-1-train-loss-2.0841-val-loss-2.1685
simplet5-epoch-2-train-loss-1.8849-val-loss-2.1693
simplet5-epoch-3-train-loss-1.7059-val-loss-2.2194
simplet5-epoch-4-train-loss-1.546-val-loss-2.2447


In [20]:
model.load_model("t5","outputs/simplet5-epoch-0-train-loss-2.3749-val-loss-2.1523",use_gpu=True)

In [21]:
text_to_summarize="""summarize: Business acquisitions agreements can involve
complex and time-consuming processes that are often costly
and involve multiple parties and intermediaries. The paper
aims to design and implement a smart contract-based system
for M&A agreements, incorporating features such as escrow
accounts, dispute resolution mechanisms, and regulatory
compliance checks. The research conducted in this study
involves researching existing M&A processes and
regulations, designing, developing and testing the system,
and evaluating its performance and ef ectiveness.
Demonstration of the potential benefits and drawbacks of
using smart contracts in M&A agreements, such as increased
ef iciency, reduced costs, increased transparency, and
reduced legal risks is done. Overall, this paper contributes to
the development of innovative blockchain-based solutions for
the business world.
 """

model.predict(text_to_summarize)

['Smart Contract-Based M&A Agreements']

In [22]:
text_to_summarize="""summarize: Skin lesions in photographs are used to detect 
melanoma, a kind of skin cancer. In order to visually identify 
the lesion from the image, an effective improved algorithm is 
needed because of the lesion's uneven shape and border. Based 
on the image's color, texture, form, and pixel intensity, the skin 
lesion is retrieved. There exists a huge number of image 
segmentation algorithms and finding and implementing an 
appropriate algorithm for a diagnosis is a tedious and timeconsuming process. Moreover, it is imperative to detect and 
treat any type of cancer in their early stages. In order to tackle 
this issue, we compared different architectures to find out which 
algorithm is the most efficient and brings about the best 
accuracy. We evaluated the approaches with Precision, Recall, 
F1 Score, accuracy and IOU score. We compared the 
performance of our approach with MIFNet, UNet, SegNet and 
FCN architectures, using the ISIC 2018 dataset. The 
comparison reveals that the MIFNet model outperforms SegNet, 
UNet and FCN architectures, with a precision of 98%
 """

model.predict(text_to_summarize)



['A Comparison of Image Segmentation Algorithms for Melanoma Detection']